# Models 

- eval(train, test, 16, 64*4, 2, device, 0.001)
    - batch_size=100, num_epochs=1000, print_point=5
- eval(train, test, 16, 64*5, 2, device, 0.001) 2

In [31]:
import torch
import torch.nn as nn
from utils.dataset import CSVDataset, CSVDatasetGroupSplit
from torch.utils.data import Dataset, DataLoader
import math
import numpy as np

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, hidden_size*2)
        self.relu2 = nn.ReLU()  
        self.l3 = nn.Linear(hidden_size*2, num_classes)

    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out
    

# Load Datasets Train
train, test = CSVDatasetGroupSplit('./data/dataframes/all_bands.csv').getSplit()

def eval(train, test, input_size, hidden_size, num_classes, device, learning_rate, batch_size=100, num_epochs=10000, print_point=10):
    # Set Params
    total_samples = len(train)
    n_iterations = math.ceil(total_samples/batch_size)

    # Data loader
    loader = DataLoader(dataset=train,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=1)

    model = NeuralNet(input_size, hidden_size, num_classes).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(loader):
              
            #inputs = inputs.reshape(-1, input_size)
            inputs = inputs.to(device)
            labels = labels.squeeze().type(torch.LongTensor).to(device)

            #print(labels.shape)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % print_point == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_iterations}], Loss: {loss.item():.4f}')
                #print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')

    PATH = './models/asd_16_ffn_4.pth'
    torch.save(model.state_dict(), PATH)

    # Data loader Test
    loader = DataLoader(dataset=test,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=1) 
    
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.squeeze().type(torch.LongTensor).to(device)
            outputs = model(inputs)     
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            acc = 100.0 * n_correct / n_samples
            print(f'Accuracy of the network on the 10000 test images: {acc} %')

# 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# train, test, input_size, hidden_size, num_classes, device, learning_rate
eval(train, test, 16, 64*4, 2, device, 0.001)

Epoch [1/10000], Step [10/41], Loss: 0.7733
Epoch [1/10000], Step [20/41], Loss: 0.8133
Epoch [1/10000], Step [30/41], Loss: 0.9033
Epoch [1/10000], Step [40/41], Loss: 0.8833
Epoch [2/10000], Step [10/41], Loss: 0.8633
Epoch [2/10000], Step [20/41], Loss: 0.7633
Epoch [2/10000], Step [30/41], Loss: 0.8833
Epoch [2/10000], Step [40/41], Loss: 0.8433
Epoch [3/10000], Step [10/41], Loss: 0.7933
Epoch [3/10000], Step [20/41], Loss: 0.8533
Epoch [3/10000], Step [30/41], Loss: 0.7733
Epoch [3/10000], Step [40/41], Loss: 0.8133
Epoch [4/10000], Step [10/41], Loss: 0.7733
Epoch [4/10000], Step [20/41], Loss: 0.8333
Epoch [4/10000], Step [30/41], Loss: 0.8233
Epoch [4/10000], Step [40/41], Loss: 0.8633
Epoch [5/10000], Step [10/41], Loss: 0.8333
Epoch [5/10000], Step [20/41], Loss: 0.8433
Epoch [5/10000], Step [30/41], Loss: 0.7833
Epoch [5/10000], Step [40/41], Loss: 0.8433
Epoch [6/10000], Step [10/41], Loss: 0.8033
Epoch [6/10000], Step [20/41], Loss: 0.8133
Epoch [6/10000], Step [30/41], L

KeyboardInterrupt: 